## Run SimSS simulations and plot the results
Version 0.1
(c) Vincent M. Le Corre (https://github.com/VMLC-PV)

In [ ]:
# Package import
%matplotlib inline
# comment the next line if you are on the jupyterhub server
%matplotlib widget 
# %matplotlib notebook
import os,platform,warnings
import numpy as np
import matplotlib.pyplot as plt
# Import homemade package by VLC
# sys.path.append('../') # comment out if the Notebook is in the Notebooks folder
from boar.SIMsalabim_utils.RunSim import PrepareSimuInputs, RunSimulation # Run simulation
from boar.SIMsalabim_utils.SimSS_plots import PlotJV, PlotJVPerf,PlotNrjDiagSimSS,PlotDensSimSS # Plotting functions
from boar.SIMsalabim_utils.CleanFolder import clean_up_output, clean_file_type # Cleaning functions
from boar.SIMsalabim_utils import plot_settings_screen # Plot settings
 
## General Inputs
warnings.filterwarnings("ignore")           # Don't show warnings
system = platform.system()                  # Operating system
max_jobs = os.cpu_count()-2                 # Max number of parallel simulations (for number of CPU use: os.cpu_count() )
do_multiprocessing = True                      # Use multiprocessing
if system == 'Windows':                     # cannot easily do multiprocessing in Windows
        max_jobs = 1
        do_multiprocessing = False
        try:                                # kill all running jobs to avoid conflicts
            os.system('taskkill.exe /F /IM simss.exe')
        except:
            pass

# Path to SIMsalabim
# path2simu = os.path.join(os.getcwd() ,'SIMsalabim','SimSS')
path2simu = '/home/vlc/Desktop/SIMsalabim/SimSS'


In [ ]:
# Prepare the command strings to run
fixed_str = '-OutPutRatio 1 -Vstep 0.025'
parameters = []
Gfrac = list(np.geomspace(1e-2,1,3))
parameters.append({'name':'Gfrac','values':Gfrac})

Simulation_Inputs = PrepareSimuInputs(path2simu,parameters=parameters,fixed_str=fixed_str,CodeName = 'SimSS',verbose=True) # creates all necessary list inputs

RunSimulation(Simulation_Inputs,max_jobs=max_jobs,do_multiprocessing=do_multiprocessing,verbose=True) # runs the simulations


In [ ]:
# Make JV plots
str_lst,JV_files,Var_files,scPars_files,code_name_lst,path_lst,labels = Simulation_Inputs
colors = plt.cm.viridis(np.linspace(0,1,max(len(str_lst),3)+1)) # prepare color for plots

PlotJV(JV_files, labels=labels,colors=colors,num_fig=0,plot_type=0,x='Vext',y=['Jext'],legend=False,xlimits=[0,1.4],ylimits=[-25,1e1],x_unit='V',y_unit='mA/cm^2',save_fig=True,fig_name=os.path.join(path2simu,'JV.png'),mark='',line_type = ['-'])


In [ ]:
# Plot performance
PlotJVPerf(Gfrac, scPars_files,y='PCE',Gfrac=Gfrac,plot_type=1,xlimits=[],ylimits=[],xlabel='Light Intensity [suns]',ylabel='PCE [%]',legend='',num_fig=1,mark='o',line_type = ['-'],norm_plot=False,fig_name=os.path.join(path2simu,'JV_perf.png'))


In [ ]:
# Make Energy diagram plots
PlotNrjDiag = True
if PlotNrjDiag:
    PlotNrjDiagSimSS([Var_files[-1]],labels,path2simu,colors=[],num_fig=2,Vext='nan',Background_color=True,show_axis=True,legend=True,fig_name=os.path.join(path2simu,'energy_diagram.jpg'),save_fig=False)


In [ ]:
# Make density plots
PlotDens = True
if PlotDens:
    # plot one file at all voltages
    PlotDensSimSS([Var_files[0]],labels,colors,num_fig=3,Vext=['all'],y=['n','p'],xlimits=[],ylimits=[1e8,1e22],x_unit='nm',y_unit='cm^-3',plot_type=2,colorbar_type='lin',colorbar_display=True,line_type = ['-','--'],legend=True,save_fig=True,fig_name=os.path.join(path2simu,'density.jpg'),verbose=False)

    # plot all files at one voltage
    colors = plt.cm.viridis(np.linspace(0,1,max(len(str_lst),3)+1))

    PlotDensSimSS(Var_files,labels,colors=colors,num_fig=4,Vext=[0.6],y=['n','p'],xlimits=[],ylimits=[1e8,1e22],x_unit='nm',y_unit='cm^-3',plot_type=2,colorbar_type='lin',colorbar_display=False,line_type = ['-','--'],legend=True,save_fig=True,fig_name=os.path.join(path2simu,'density.jpg'),verbose=False)


In [ ]:
# Clean output files from simulation folders
Do_Cleaning = True # Careful, this will delete all files in the folder
if Do_Cleaning:
    clean_up_output('tj',path2simu)
    clean_up_output('tVG',path2simu)
    clean_up_output('JV',path2simu)
    clean_up_output('Var',path2simu)
    clean_up_output('scPars',path2simu)
    clean_up_output('log',path2simu)
    clean_up_output('Str4Parallel',path2simu)
    clean_up_output('joblog',path2simu)
    clean_file_type('.jpg',path2simu) # clean all jpg files
    clean_file_type('.png',path2simu) # clean all png files
    